In [68]:
import os
import sys
import numpy as np
import json
import torch

ImportError: No module named 'torch'

In [3]:
### path and parameter
path_data = './MLDS_hw2_data/'


In [18]:
### load data

### load *.npy
lst_name_train_npy = os.listdir('{}training_data/feat'.format(path_data))
lst_name_test_npy = os.listdir('{}testing_data/feat'.format(path_data))

lst_train_npy = []
for npy in lst_name_train_npy :
    lst_train_npy += [np.load('{}training_data/feat/{}'.format(path_data, npy))]
lst_test_npy = []
for npy in lst_name_test_npy :
    lst_test_npy += [np.load('{}testing_data/feat/{}'.format(path_data, npy))]

print ('len of lst_train_npy : '+str(len(lst_train_npy))+'\nshape of train npy : '+str(lst_train_npy[0].shape))
print ('len of lst_test_npy : '+str(len(lst_test_npy))+'\nshape of test npy : '+str(lst_test_npy[0].shape))

### load training_label.json testing_label.json
with open('{}training_label.json'.format(path_data)) as f :
    lst_dict_label_train = json.load(f)
print ('\ntraining_label.json : ')
print ('caption : \n' + str(lst_dict_label_train[0]['caption']))
print ('id : \n' + str(lst_dict_label_train[0]['id']))
    

len of lst_train_npy : 1450
shape of train npy : (80, 4096)
len of lst_test_npy : 100
shape of test npy : (80, 4096)

training_label.json : 
caption : 
['A woman goes under a horse.', 'A woman crawls under a horse and gets a surprise.', 'A girl is jutting out her head from between the back legs of a horse who shits on her head.', "A horse defecated on a woman's head.", 'A horse excretes on the head of a woman as she gets in between the fore anf hind legs and sticks her head out.', "A horse is defecating on a woman's head.", 'A horse poops on a woman.', 'A horse poops on a woman.', "A lady was pooped on because she was under a horse's tail end.", 'A woman crawls under a horse and when she sticks her head between the horses rear legs the horse defecates on her head.', "A woman goes under a horse's behind and gets pooped on.", 'A woman goes underneath a horse.', 'A woman is crawling under a horse.', 'A woman is getting pooped on by a horse.', 'A woman is walking between horse legs.', 'A w

In [66]:
### build dict_map_W2I and dict_map_I2W to mapping from word to index

### count unique words
lst_word = list(set([word for dict_label_train in lst_dict_label_train for str_label in dict_label_train['caption'] for word in str_label[:-1].split()]))
lst_word += ['<BOS>']
lst_word += ['<EOS>']
n_class = len(lst_word)
print ('len(lst_word) : {}'.format(n_class))

### map from word to index
dict_map_W2I = dict()
index = 0
for word in lst_word :
    if word not in dict_map_W2I :
        dict_map_W2I[word] = index
        index += 1
        
### map from index to word
dict_map_I2W = {v:k for k,v in dict_map_W2I.items()}

print ('<<-- dict_map_W2I and dict_I2W are built completely -->>')

len(lst_word) : 6871
<<-- dict_map_W2I and dict_I2W are built completely -->>


In [67]:
### index to one-hot
def Str2OneHot(sentence, n_class) :
    ### sentence to lst_index_sentence
    sentence = '<BOS> ' + sentence[:-1] + ' <EOS>'
    lst_word = sentence.split()
    ary_oneHot = np.zeros((len(lst_word),n_class))
    lst_index_word = [dict_map[word] for word in lst_word]
    ary_oneHot[range(len(lst_word)),lst_index_word] = 1
    return ary_oneHot
    
### just for test
Str2OneHot('A woman goes under a horse.', n_class)
        

array([[ 0.,  0.,  0., ...,  0.,  1.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       ..., 
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  1.]])

In [ ]:
### note
# remember to add <BOS> and <EOS>
# Tom's should split as "TOM" and "s"
# notice [:-1] to delete "."